In [125]:
import re
import pandas as pd
from nltk.corpus import stopwords

import copy
import numpy as np

import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [126]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\krati (2).xlsx")
news.head()

,Title,Body,Category,Old MegaCategory,MegaCategory,important,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial,Financial,Financial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial,Financial,Financial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial,Financial,Financial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial,Financial,Financial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial,Financial,Financial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['am', 'pm'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [128]:
news['text'] = str(news.Title) + "  "+ str(news.Body)

In [129]:


news['clean_text'] = news['text'].apply(remove_stop_words) 

#remove_stop_words_Body


In [130]:
# ## Split data
print("\nSplitting data")
title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_text'], news.important	,test_size =.1)
title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.1)

print("Training: ",len(title_tr))
      
print("Testing: ",len(title_te))


Splitting data
Training:  2025
Testing:  250


In [131]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [132]:
print(Ytr)

[1 2 4 ... 4 4 4]


In [133]:
Xtr

<2025x519 sparse matrix of type '<class 'numpy.int64'>'
	with 1050975 stored elements in Compressed Sparse Row format>

In [134]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)

dc_pred = dc.predict(Xde)



Training baseline classifier


In [135]:
print(classification_report(Yde, dc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dc_pred == Yde)))

                    precision    recall  f1-score   support

         Financial       0.04      0.04      0.04        25
            Growth       0.18      0.11      0.14        27
     Negative News       0.41      0.45      0.43        89
 Stock Performance       0.41      0.45      0.43        89
Uncategorised News       0.20      0.23      0.21        31
            others       0.04      0.04      0.04        25

          accuracy                           0.34    106972
         macro avg       0.25      0.26      0.25    106972
      weighted avg       0.33      0.35      0.34    106972

Accuracy achieved is 0.2577777777777778


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))


In [136]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
dt_pred = dt.predict(Xde)
print(classification_report(Yde, dt_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(dt_pred == Yde)))

Training Decision tree
                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        25
            Growth       0.00      0.00      0.00        27
     Negative News       0.40      1.00      0.57        89
 Stock Performance       0.40      1.00      0.57        89
Uncategorised News       0.00      0.00      0.00        31
            others       0.00      0.00      0.00        25

          accuracy                           0.50    106972
         macro avg       0.16      0.41      0.23    106972
      weighted avg       0.28      0.70      0.39    106972

Accuracy achieved is 0.39555555555555555


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [137]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        25
            Growth       0.00      0.00      0.00        27
     Negative News       0.40      1.00      0.57        89
 Stock Performance       0.40      1.00      0.57        89
Uncategorised News       0.00      0.00      0.00        31
            others       0.00      0.00      0.00        25

          accuracy                           0.50    106972
         macro avg       0.16      0.41      0.23    106972
      weighted avg       0.28      0.70      0.39    106972

Accuracy achieved is 0.39555555555555555


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [138]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_nb == Yde)))

Training Multinomial Naive Bayesian
                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        25
            Growth       0.00      0.00      0.00        27
     Negative News       0.40      1.00      0.57        89
 Stock Performance       0.40      1.00      0.57        89
Uncategorised News       0.00      0.00      0.00        31
            others       0.00      0.00      0.00        25

          accuracy                           0.50    106972
         macro avg       0.16      0.41      0.23    106972
      weighted avg       0.28      0.70      0.39    106972

Accuracy achieved is 0.39555555555555555


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [139]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
svc_pred = svc.predict(Xde)
print(classification_report(Yde, svc_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(svc_pred == Yde)))

Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        25
            Growth       0.00      0.00      0.00        27
     Negative News       0.40      1.00      0.57        89
 Stock Performance       0.40      1.00      0.57        89
Uncategorised News       0.00      0.00      0.00        31
            others       0.00      0.00      0.00        25

          accuracy                           0.50    106972
         macro avg       0.16      0.41      0.23    106972
      weighted avg       0.28      0.70      0.39    106972

Accuracy achieved is 0.39555555555555555


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [140]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
mlp_pred = mlp.predict(Xde)
print(classification_report(Yde, mlp_pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(mlp_pred == Yde)))

Training Multilayered Perceptron
                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        25
            Growth       0.00      0.00      0.00        27
     Negative News       0.40      1.00      0.57        89
 Stock Performance       0.40      1.00      0.57        89
Uncategorised News       0.00      0.00      0.00        31
            others       0.00      0.00      0.00        25

          accuracy                           0.50    106972
         macro avg       0.16      0.41      0.23    106972
      weighted avg       0.28      0.70      0.39    106972

Accuracy achieved is 0.39555555555555555


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2025, does not match size of target_names, 6
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [141]:
print('Accuracy achieved is ' + str(np.mean(dc_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(dt_pred   == Yde)))
print('Accuracy achieved is ' + str(np.mean(pred_nb   == Yde)))
print('Accuracy achieved is ' + str(np.mean(svc_pred  == Yde)))
print('Accuracy achieved is ' + str(np.mean(mlp_pred  == Yde)))

Accuracy achieved is 0.2577777777777778
Accuracy achieved is 0.39555555555555555
Accuracy achieved is 0.39555555555555555
Accuracy achieved is 0.39555555555555555
Accuracy achieved is 0.39555555555555555


In [142]:
# # Final Model: Multilayered Perceptron
# ## Predict test data
print("\n\nPredicting test data using Multilayered Perceptron")
pred_final = mlp.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(accuracy_score(Yte,pred_final)))



Predicting test data using Multilayered Perceptron
                    precision    recall  f1-score   support

         Financial       0.00      0.00      0.00        31
            Growth       0.00      0.00      0.00        21
     Negative News       0.43      1.00      0.60       108
 Stock Performance       0.43      1.00      0.60       108
Uncategorised News       0.00      0.00      0.00        40
            others       0.00      0.00      0.00        31

          accuracy                           0.54    125227
         macro avg       0.18      0.41      0.25    125227
      weighted avg       0.31      0.72      0.44    125227

Accuracy achieved is 0.432


In [143]:
output = {"title":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}

In [144]:
#df = pd.DataFrame(output, columns=["title","predicted","actual"])
#df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_title_predication_1.csv")

In [145]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"text":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["text","predicted","actual"])


Accuracy achieved is 0.432


In [218]:
df = pd.DataFrame(output, columns=["text","predicted","actual"])
df.to_csv("C:\\Users\\Kratika\\Downloads\\News_growth.csv")